# Cosmic Ray Energy Spectrum 

<i>Notebook based on the Energy spectrum notebook from the Pierre Auger Observatory Open Data release 2021 (<a href="https://doi.org/10.5281/zenodo.4487613">DOI 10.5281/zenodo.4487613</a>) and [web site](http://www.auger.org/opendata/), as well as the [KCDC spectra files](http://kcdc.ikp.kit.edu/spectra/) (need to create an account to access).</i>

In [ ]:
![ -d /kaggle/input/release-2021-v1/augeropendata ] && [ ! -d augeropendata ] && ln -s /kaggle/input/release-2021-v1/augeropendata augeropendata  # kaggle specific linking dataset to augeropendata directory
# Data loading, encapsulated to make it less installation and OS dependant
import os.path
from zipfile import ZipFile
def AugerOpen(fdir, file):
    """
    Loads a file from the auger open data release. Can be either in the local directory,
    in the parent directory or in the augeropendata directory.
    File is identified by it directory *fdir* and filename *file* and can be found in the directory
    or in a zip file.
    """
    for loc in [".", "..", "augeropendata", "data"]:
        fname = os.path.join(loc, fdir, file)
        if os.path.isfile(fname):
            return open(fname)
        zname=os.path.join(loc, fdir + ".zip")
        if os.path.isfile(zname):
            with ZipFile(zname) as myzip:
                return myzip.open(os.path.join(fdir, file))
    raise FileNotFoundError(os.path.join(fdir, file))

In [ ]:
# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [18, 9]
plt.rcParams["font.size"] = 20

In [ ]:
# energy cut for these data (not indicated in the website), preparing data and getting exposure
energy_cut=10**0.4
data = pd.read_csv(AugerOpen("summary", "dataSummary.csv"))
data = data[(data["sdStandard"]>0)&(data["sd_energy"]>energy_cut)]
data = data.sort_values(by='sdid')
energy = data.drop_duplicates('id')["sd_energy"] 
exposure = data["sd_exposure"].iat[-1]
print(f'The exposure of the Auger public data is {exposure:7.1f} km2 sr year.')

In [ ]:
# making bins and histogram
logx = np.linspace(18, 20.5, 26)
bins = pow(10, logx);
y,x = np.histogram(1e18*energy, bins)
dx=x[1:]-x[:-1]
x=np.sqrt(x[1:]*x[:-1])

In [ ]:
# first "histogram" plot
plt.plot(x, y/exposure, marker="o",linestyle='None')
plt.xscale("log")
plt.yscale("log")
plt.xlabel('E [eV]')
plt.ylabel(r'Events [km$^{-2}$ sr$^{-1}$ yr$^{-1}$]')

for E, count in zip(x, y):
    if count > 0:
        plt.annotate(count, (E*1.05, count/exposure), va='bottom')

In [ ]:
# Eˆ3 x Flux plot with (bad) error bars
plt.errorbar(x, x**3*y/exposure/dx, x**3*np.sqrt(y)/dx/exposure,fmt='D',markersize=7)
plt.xscale("log")
plt.yscale("log")
plt.xlabel('E [eV]')
plt.ylabel(r'E$^3$ x Flux [km$^{-2}$ sr$^{-1}$ yr$^{-1}$ eV$^2$]')
plt.show()

In [ ]:
# getting spectra from other experiments (via https://kcdc.ikp.kit.edu/spectra/ and plotting all of them)
t=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_tibet.txt")
k=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_k.txt")
kg=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_kg.txt")
a=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_auger.txt")
a08=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_auger_PRL2008.txt")
a10=pd.read_csv("/kaggle/input/kaskade-grande-basic-spectrum/spec_auger_PLB2010.txt")
ms2kmyr=1e6*86400*365.25 # kcdc data are in m^-2 s^-1 (not indicated on the website)
for i in ["Flux","UncertLow","UncertHigh"]:
    for d in [k,a,a08,a10,kg,t]:
        d[i]=d[i]*ms2kmyr
        d["Flux"][d["UncertLow"]==0]=0
plt.errorbar(x, y/exposure/dx, np.sqrt(y)/dx/exposure,fmt='D',markersize=9,label="Auger public data 2021")
plt.errorbar(t["E"], t["Flux"], [t["UncertLow"],t["UncertHigh"]],fmt='o',label="Tibet III")
plt.errorbar(k["E"], k["Flux"], [k["UncertLow"],k["UncertHigh"]],fmt='o',label="Kaskade")
plt.errorbar(kg["E"], kg["Flux"], [kg["UncertLow"],kg["UncertHigh"]],fmt='o',label="Kaskade Grande")
plt.errorbar(a["E"], a["Flux"], [a["UncertLow"],a["UncertHigh"]],fmt='o',label="Auger ICRC 2017")
#plt.errorbar(a08["E"], a08["Flux"], [a08["UncertLow"],a08["UncertHigh"]],fmt='o',label="Auger PRL 2008")
#plt.errorbar(a10["E"], a10["Flux"], [a10["UncertLow"],a10["UncertHigh"]],fmt='o',label="Auger PLB 2010")
plt.xscale("log")
plt.yscale("log")
plt.xlabel('E [eV]')
plt.ylabel(r'Flux [km$^{-2}$ sr$^{-1}$ yr$^{-1}$ eV$^{-1}$]')
plt.plot([1e14,5e20],1e38*np.array([1e14,5e20])**-3,linestyle=':', label="E$^{-3}$")
plt.legend()
plt.show()

In [ ]:
# gPlot x E^3
if a["Flux"][0]<1: # so it only does once if cell is rerun many times
    for i in ["Flux","UncertLow","UncertHigh"]:
        for d in [k,a,a08,a10,kg,t]:
            d[i]=d[i]*d["E"]**3
plt.errorbar(x, x**3*y/exposure/dx, x**3*np.sqrt(y)/dx/exposure,fmt='o',label="Auger public data 2021",marker='D',markersize=7)
plt.errorbar(t["E"], t["Flux"], [t["UncertLow"],t["UncertHigh"]],fmt='o',label="Tibet III")
plt.errorbar(k["E"], k["Flux"], [k["UncertLow"],k["UncertHigh"]],fmt='o',label="Kaskade")
plt.errorbar(kg["E"], kg["Flux"], [kg["UncertLow"],kg["UncertHigh"]],fmt='o',label="Kaskade Grande",alpha=0.3)
plt.errorbar(a["E"], a["Flux"], [a["UncertLow"],a["UncertHigh"]],fmt='o',label="Auger ICRC 2017")
#plt.errorbar(a08["E"], a08["Flux"], [a08["UncertLow"],a08["UncertHigh"]],fmt='o',label="Auger PRL 2008")
#plt.errorbar(a10["E"], a10["Flux"], [a10["UncertLow"],a10["UncertHigh"]],fmt='o',label="Auger PLB 2010")
plt.errorbar(kg["E"], kg["Flux"]*1.3, [kg["UncertLow"]*1.3,kg["UncertHigh"]*1.3],fmt='o',label="Kaskade Grande (+30%)",c='C3')
plt.xscale("log")
plt.yscale("log")
plt.xlabel('E [eV]')
plt.ylabel(r'E$^3$ x Flux [km$^{-2}$ sr$^{-1}$ yr$^{-1}$ eV$^{2}$]')
plt.legend()
plt.show()